## **데이콘 뉴스 기사 레이블 복구 해커톤**

본 프로젝트는 뉴스 데이터셋의 'category'필드를 복구하는 것을 목적으로 한다.  
데이터셋은 6개의 카테고리로 분류되어야 하며, 6만 행으로 이루어진 csv 파일이다. 파일에는 'id'와 'text'필드만 있다.  
복구해야 하는 카테고리 종류는 다음과 같다.  
0: Business  
1: Entertainment  
2: Politics  
3: Sports  
4: Tech  
5: World


-------------------
Dataset Info.  
news.csv  
id : 샘플 고유 id  
title : 뉴스 기사 제목  
content : 뉴스 기사 전문  

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import save_model, load_model
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
data = pd.read_csv("news.csv")
data_with_labels = pd.read_csv("news_added_labels.csv")

In [ ]:
data

,id,title,contents
0,NEWS_00000,Spanish coach facing action in race row,MADRID (AFP) - Spanish national team coach Lui...
1,NEWS_00001,Bruce Lee statue for divided city,"In Bosnia, where one man #39;s hero is often a..."
2,NEWS_00002,Only Lovers Left Alive's Tilda Swinton Talks A...,Yasmine Hamdan performs 'Hal' which she also s...
3,NEWS_00003,Macromedia contributes to eBay Stores,Macromedia has announced a special version of ...
4,NEWS_00004,Qualcomm plans to phone it in on cellular repairs,Over-the-air fixes for cell phones comes to Qu...
...,...,...,...
59995,NEWS_59995,"Dolphins Break Through, Rip Rams For First Win",But that #39;s OK. Because after a 31-14 rout ...
59996,NEWS_59996,"After Steep Drop, Price of Oil Rises",The freefall in oil prices ended Monday on a s...
59997,NEWS_59997,Pro football: Culpepper puts on a show,To say Daunte Culpepper was a little frustrate...
59998,NEWS_59998,Albertsons on the Rebound,The No. 2 grocer reports double-digit gains in...


In [ ]:
# 600개 기사에 수기 labeling
data_with_labels

,id,title,contents,category
0,NEWS_00000,Spanish coach facing action in race row,MADRID (AFP) - Spanish national team coach Lui...,3
1,NEWS_00001,Bruce Lee statue for divided city,"In Bosnia, where one man #39;s hero is often a...",1
2,NEWS_00002,Only Lovers Left Alive's Tilda Swinton Talks A...,Yasmine Hamdan performs 'Hal' which she also s...,1
3,NEWS_00003,Macromedia contributes to eBay Stores,Macromedia has announced a special version of ...,4
4,NEWS_00004,Qualcomm plans to phone it in on cellular repairs,Over-the-air fixes for cell phones comes to Qu...,4
...,...,...,...,...
595,NEWS_00595,Ward KOs 8-year drought,It #39;s been eight years since a US boxer sto...,3
596,NEWS_00596,Trump Signs Bill Making It Easier For Employer...,Republicans just repealed a major safety regul...,2
597,NEWS_00597,Gymnastics #39; pettiness is all-around,"ATHENS, Greece -- If international gymnastics ...",3
598,NEWS_00598,Group: Israel Violating International Law (AP),AP - Israel has violated international law by ...,5


In [ ]:
!pip install nltk

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# 데이터 전처리 함수 정의
def preprocessing(text):
    # 소문자 변환, 특수문자 제거
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)

    # 표제어 추출
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    text = ' '.join([lemmatizer.lemmatize(word) for word in words])

    # 불용어 제거
    stop_words = set(stopwords.words('english'))
    words = text.split()
    text = ' '.join([word for word in words if word not in stop_words])

    return text

In [ ]:
# 수기 레이블링된 데이터 제외하여 슬라이스(추론 데이터)
data = data.iloc[600:]

In [ ]:
# 추론 데이터 전처리
data['title'] = data['title'].apply(preprocessing)
data['contents'] = data['contents'].apply(preprocessing)

# 훈련 데이터 전처리
data_with_labels['title'] = data_with_labels['title'].apply(preprocessing)
data_with_labels['contents'] = data_with_labels['contents'].apply(preprocessing)

<ipython-input-9-8b8146d3b450>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['title'] = data['title'].apply(preprocessing)
<ipython-input-9-8b8146d3b450>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['contents'] = data['contents'].apply(preprocessing)


In [ ]:
# 추론 데이터 병합
data['combined_text'] = data['title'] + ' ' + data['contents']

# 훈련 데이터 병합
data_with_labels['combined_text'] = data_with_labels['title'] + ' ' + data_with_labels['contents']

<ipython-input-10-987270d3db3f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['combined_text'] = data['title'] + ' ' + data['contents']


In [ ]:
data['combined_text']

600      u2019t let trump congress tax public infrastru...
601      e voting sceptic use web monitor election tech...
602      john kelly told lawmaker trump build wall alon...
603      hp 39 compaq x pc us new pentium 4 hewlett pac...
604      football site set miaa football committee yest...
                               ...                        
59995    dolphin break rip ram first win 39 ok 31 14 ro...
59996    steep drop price oil rise freefall oil price e...
59997    pro football culpepper put show say daunte cul...
59998    albertsons rebound 2 grocer report double digi...
59999    cassini craft spy saturn moon dione ap ap cass...
Name: combined_text, Length: 59400, dtype: object

In [ ]:
data_with_labels['combined_text']

0      spanish coach facing action race row madrid af...
1      bruce lee statue divided city bosnia one man 3...
2      lover left alive tilda swinton talk almost qui...
3      macromedia contributes ebay store macromedia h...
4      qualcomm plan phone cellular repair air fix ce...
                             ...                        
595    ward ko 8 year drought 39 eight year since u b...
596    trump sign bill making easier employer hide wo...
597    gymnastics 39 pettiness around athens greece i...
598    group israel violating international law ap ap...
599    u2019s time democrat get trump train author sp...
Name: combined_text, Length: 600, dtype: object

In [ ]:
# 토크나이저와 패딩
max_words = 10000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)

# 추론 데이터 토큰화
tokenizer.fit_on_texts(data['combined_text'])
data_sequences = tokenizer.texts_to_sequences(data['combined_text'])
data_padded_sequences = pad_sequences(data_sequences, maxlen=max_len, padding='post')

# 훈련 데이터 토큰화
tokenizer.fit_on_texts(data_with_labels['combined_text'])
data_with_labels_sequences = tokenizer.texts_to_sequences(data_with_labels['combined_text'])
data_with_labels_padded_sequences = pad_sequences(data_with_labels_sequences, maxlen=max_len, padding='post')

In [ ]:
labels = data_with_labels['category']

## **모델링**

1D 합성곱 레이어와 GlobalMaxPooling1D 레이어를 사용하여 텍스트 데이터를 처리하고, 그 후에 Dense 레이어를 추가하여 최종 분류를 수행

In [ ]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data_with_labels_padded_sequences, labels, test_size=0.2, random_state=1028)

# 모델 정의
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))  # 6개의 클래스 출력 레이어

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit(X_train, y_train, epochs=50, batch_size=128)

# 모델 평가
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"테스트 정확도: {test_acc}")


Epoch 1/50
4/4 [==============================] - 2s 266ms/step - loss: 1.7841 - accuracy: 0.2021
Epoch 2/50
4/4 [==============================] - 1s 272ms/step - loss: 1.7274 - accuracy: 0.3000
Epoch 3/50
4/4 [==============================] - 1s 272ms/step - loss: 1.6771 - accuracy: 0.3417
Epoch 4/50
4/4 [==============================] - 1s 273ms/step - loss: 1.6256 - accuracy: 0.4104
Epoch 5/50
4/4 [==============================] - 1s 268ms/step - loss: 1.5840 - accuracy: 0.4750
Epoch 6/50
4/4 [==============================] - 1s 268ms/step - loss: 1.5345 - accuracy: 0.5146
Epoch 7/50
4/4 [==============================] - 1s 323ms/step - loss: 1.4666 - accuracy: 0.5875
Epoch 8/50
4/4 [==============================] - 2s 493ms/step - loss: 1.3870 - accuracy: 0.6687
Epoch 9/50
4/4 [==============================] - 1s 284ms/step - loss: 1.2910 - accuracy: 0.6896
Epoch 10/50
4/4 [==============================] - 1s 262ms/step - loss: 1.1976 - accuracy: 0.7063
Epoch 11/50
4/4 [==

In [ ]:
# 모델 저장
model.save('trained_model.h5')

# 모델 불러오기
loaded_model = load_model('trained_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
data_padded_sequences

array([[2239,  454,   13, ...,    0,    0,    0],
       [ 476, 1063,  273, ...,    0,    0,    0],
       [ 215, 2085,  319, ...,    0,    0,    0],
       ...,
       [ 993,  335,  246, ...,    0,    0,    0],
       [9845, 1726,   35, ...,    0,    0,    0],
       [3057, 4186, 2651, ...,    0,    0,    0]], dtype=int32)

In [ ]:
# 추론하기
submission_df = pd.read_csv('sample_submission.csv')

all_predictions = model.predict(data_padded_sequences)

# 'sample_submission.csv' 파일의 'category' 열에 결과 저장
submission_df['category'].iloc[600:] = np.argmax(all_predictions, axis=1)
submission_df.to_csv('result_submission.csv', index=False)


1857/1857 [==============================] - 45s 24ms/step


<ipython-input-28-c639a9be3f85>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_df['category'].iloc[600:] = np.argmax(all_predictions, axis=1)


In [ ]:
result_df = pd.read_csv('result_submission.csv')

In [ ]:
result_df

,id,category
0,NEWS_00000,-1
1,NEWS_00001,-1
2,NEWS_00002,-1
3,NEWS_00003,-1
4,NEWS_00004,-1
...,...,...
59995,NEWS_59995,3
59996,NEWS_59996,0
59997,NEWS_59997,0
59998,NEWS_59998,2


In [ ]:
data_with_labels['category']

0      3
1      1
2      1
3      4
4      4
      ..
595    3
596    2
597    3
598    5
599    2
Name: category, Length: 600, dtype: int64

In [ ]:
result_df['category'].iloc[:600] = data_with_labels['category']
result_df.to_csv('final_submission.csv', index=False)

<ipython-input-33-3b8de47e766e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['category'].iloc[:600] = data_with_labels['category']
